# Metrics, tensors and all that

## Getting started

Execute the following line of code to initialise the system. Just click in the cell and press shift-enter.

On a mobile device, there may instead be a 'play' button next to the cell. You may want to ask your browser for the desktop version, which allows for zooming parts of the screen with the usual pinch gesture.

If a * appears in the square brackets to the left of the cell, followed (fairly) shortly by a 1, then all is well.

In [ ]:
from metric import *

You should do the same for each code block as you go through this notebook. If you jump ahead, make sure that any object you use has been defined beforehand (usually with an "=Metric([...])" call.

You can also edit the code or add new code of your own. At the end are some other metrics which you might like to explore, and you can again add your own. A good source for these is [MathWorld](https:www.mathworld.com).

### Cartesian coordinates

Let's first set up the simplest coordinate system: two-dimensional Cartesian coordinates. Again, click in the following cell and press shift-enter, or press the 'play' button.

In [ ]:
cart2D = Metric([x,y])
cart2D.setDiagonal([1,1])

The first line specifies the coordinates, The second sets the coefficients of the line element - we assume that there are no cross-terms for now; that is, only the diagonal terms are non-zero.

We can inspect the matrix form of the metric. For 2D Cartesians, it's the identity matrix.

In [ ]:
cart2D.array

What would the dual metric be?

In [ ]:
cart2D.dual()

It's the identity matrix again. For all metrics - even the most complicated - we get the matrix form of the dual metric by taking the inverse of the matrix form of the metric, and the identity matrix is its own inverse. 

If our metric is *diagonal*, getting to the dual and back is even easier: each diagonal element is just the reciprocal of its counterpart.

The metric tells us how long the basis vectors are and how they are aligned with respect to one another. Theform of this metric - all 1's and 0's - tells us that the basis vectors here have length one and are a right angles. They are *orthonormal*. 

### Plane polar coordinates

We'll now look at plane polar coordinates. These describe the same flat space as Cartesians, but there are some interesting differences.

In [ ]:
planePolars = Metric([r, theta])
planePolars.setDiagonal([1,r**2])

We can check the metric we've specified.

In [ ]:
planePolars.array

You should be able to write down the matrix form of the dual metric using the rule above. The next cell checks this.

In [ ]:
planePolars.dual()

### Connection coefficients

The unit basis vectors for plane polars are $\hat{r}$ and $\hat{\theta}$. 

If we choose some point $P$ and an origin $O$, then we can draw the displacement vector $\overrightarrow{OP}$. It has length $r$ and makes an angle $\theta$ with the positive $x$-axis.

$\hat{r}$ is a unit vector in the direction of $\overrightarrow{OP}$ while $\hat{\theta}$ is at right angles to $\overrightarrow{OP}$ pointing in the direction of increasing $\theta$.

![alt text](planePolars.png "Plane polar basis vectors")



Now, what happens to the basis vectors as we move the point $P$?

Both basis vectors **change** as we move around in our space, because both are defined in terms of $\overrightarrow{OP}$ which necessarily changes when we move $P$.

As we start to look at coordinate systems in which the basis vectors are not the same everywhere, we will need something to describe **how** they change from point to point. It turns out that useful tools for doing this are the connection coefficients. (Some sources call them Christoffel symbols, and specifically Christoffel symbols of the second kind.)

Choose a basis vector labelled $j$. Consider its component in the direction of a basis vector labelled$i$. Say what the derivative is of that component with respect to the direction defined by a basis vector labelled $k$. We will write that quantity as $${\Gamma^i}_{jk}$$ where the symbol is an uppercase Greek gamma. We can calculate one for each combination of $i$, $j$ and $k$ to get the full set of connection coefficients.

Some frankly quite tedious algebra leads to an expression for each connection coefficient that we can use for calculation: 
$${\Gamma^i}_{jk} = \frac{1}{2}\sum_l{g^{il} \left ( \frac{\partial g_{lk}}{\partial x^j} + \frac{\partial g_{jl}}{\partial x^l} - \frac{\partial g_{jk}}{\partial x^l}\right )}$$

We need to start off by writing down the derivatives of each metric component with respect to each coordinate. With plane polars, only the bottom right component $r^2$ will give a non-zero derivative, and only when differentiated with respect to $r$. Remembering that our componets are labelled starting from zero, that means that the only non-zero derivative to go into the bracket above is $$\frac{\partial g_{11}}{\partial x^0}$$.

We can also save a little work by noticing that here (but not always!) the metric and dual metric are diagonal, so $g^{il}$ will be non-zero only when $i$ and $l$ are the same. That means we can drop the summation, and just replace every $l$ index we see with an $i$, leaving us with
$${\Gamma^i}_{jk} = \frac{1}{2}{g^{ii} \left ( \frac{\partial g_{ik}}{\partial x^j} + \frac{\partial g_{ji}}{\partial x^k} - \frac{\partial g_{jk}}{\partial x^i}\right )}$$.

We get a non-zero term for $i=k=1$ with $j = 0$ in the first term in the bracket, $j = i = 1$ with $k = 0$ in the second and for $j = k = 1$ with $i=0$ in the third.

Plugging these values into the equation, the first combination gives ${\Gamma^1}_{01}=\frac{1}{2}g^{11}\frac{\partial g_{11}}{\partial x^0}=\frac{1}{2}\frac{1}{r^2}2r=\frac{1}{r}$.

The second gives ${\Gamma^1}_{10}$. We could calculate this from scratch, or we can just note that the connection coefficient is always symmetric in the lower two indices, so $${\Gamma^1}_{10}={\Gamma^1}_{01}=\frac{1}{r}$$.

The final combination gives $${\Gamma^0}_{11} = -\frac{1}{2}g^{00}\frac{\partial g_{11}}{\partial x^0}= -\frac{1}{2}2r = -r$$.

Having done that once, we probably don't want to do it again. All that mechanical substituting in of values is easily programmable if we have access to a computer algebra library that can handle the derivatives.

This function gives the connection coefficients. Note that it doesn't yet display up or down indices, so just enter the top one first.

Satisfy yourself that our results are confirmed by the function.

In [ ]:
planePolars.connection(0,1,1)

## Curvature

### The Riemann curvature tensor

We would like a tensor quantity that describes the curvature at any point in the space or spacetime defined by a given metric. 

A mathematician named Riemann devised such an object, now known as the **Riemann curvature tensor**. We would expect it to involve second-order derivatives of the metric; remembering that the connection coefficients are *first*-order deivatives of the metric, indeed it does. It does look rather fearsome.

$${R^i}_{jkl} \equiv \frac{\partial {\Gamma^l}_{ik}}{\partial x^j} - \frac{\partial {\Gamma^l}_{ij}}{\partial x^k} + 
    \sum_m{{\Gamma^m}_{ik}{\Gamma^l}_{mj}} -  \sum_m{{\Gamma^m}_{ij}{\Gamma^l}_{mk}}$$
    
This tensor is actually the simplest object that can describe curvature - it has no derivatives of order greater than two - and Einstein hopefully assumed that the results it gave would be correct. That is one reason for continuing to test the predictions of GR: it might be that we will discover that it is only an approximation, and that we need to build on a curvature tensor with additional terms. To date, though, no deviations from the predictions of GR have ever been observed.

We can again program a function to do this work for us.

In [ ]:
planePolars.riemann(0,0,0,0)

What would you expect the other components of Riemann to be for plane polar coordinates? 

The space is flat so we expect zero curvature. Though there are non-zero connection coefficients, when we put these into the definition of Riemann, after much cancelling we get zero.

Riemann has a great many symmetries, and so rather a lots of its components are not independent; that is they are either identically zero, or they are plus or minus another component. In just two dimensions, there is only one independent component. Conventionally this is taken to be ${R^0}_{101}$ so we only need to check the value of this one component. If it is zero, then all the components are zero and our space(time) is flat.

In [ ]:
planePolars.riemann(0,1,0,1)

Let's set up coordinates for a space which is curved: the surface of a sphere.

In [ ]:
sphereSurface = Metric([theta, phi])
sphereSurface.setDiagonal([r**2, (r * sin(theta))**2])

Let's run our test for flatness in two dimensions.

In [ ]:
sphereSurface.riemann(0,1,0,1)

The surface of a sphere is indeed curved!

### The Ricci tensor

We can contract two indices of the Riemann curvature tensor to get another tensor, know as the **Ricci tensor**. This tells us about curvature in an infinitesimal region around a point due to sources of curvature - mass and energy - *within* that region. In a vacuum, all the element of this tensor will be zero.

S383 uses a slightly unusual convention for this contraction; the components of Ricci have the opposite sign to those you will find in most other books.

In [ ]:
sphereSurface.ricci()

We can contract again over the two indices of the Ricci tensor to get the **Ricci scalar curvature**. Again, our convention gives the opposite sign to most other texts.

In [ ]:
sphereSurface.ricciScalar()

The **Gaussian curvature** $K$ is given by $\frac{R_{0101}}{g}$ where $g$ is the determinant of the matrix form of the metric. It is half the Ricci scalar but has the opposite sign using our convention.

In [ ]:
sphereSurface.gaussianCurvature()    

## The geodesic equations

The shortest distance between two points on a curved surface will not look like a straight line to an external observer.

Looking at a map and it would appear that the shortest route between London and Los Angeles involves following a straight line that heads a little south of west.
Flights from London, though, head north towards Scotland, as the shortest distance between two points on a spherical surface is a **great circle**.

The geodesic equations are a set of coupled, second-order differential equations describing this shortest path - a **geodesic** - between two points in a space described by a given metric. They are generally difficult or impossible to solve analytically unless we have a high degree of symmetry.

There is one equation for each coordinate. Set the value of $i$ to choose a coordinate, then sum over all possible values of the other indices.

$$\frac{\textrm{d}^2 x^i}{\textrm{d} \lambda^2} + {\Gamma^i}_{jk}\frac{\textrm{d} x^j}{\textrm{d} \lambda}\frac{\textrm{d} x^k}{\textrm{d} \lambda} = 0$$

Here are the equations for the surface of a sphere. Python isn't great at formatting these...

In [ ]:
sphereSurface.geodesicEquation(0)

In [ ]:
sphereSurface.geodesicEquation(1)

What do the geodesic equations look like in Cartesians? Have you seen these equations before?

In [ ]:
cart2D = Metric([x,y])
cart2D.setDiagonal([1,1])

In [ ]:
cart2D.geodesicEquation(0)

In [ ]:
cart2D.geodesicEquation(1)

In Cartesians, the connection coefficients are zero, and we get back Newton's first law.

Let's look at plane polars.

In [ ]:
planePolars = Metric([r, theta])
planePolars.setDiagonal([1,r**2])

In [ ]:
planePolars.geodesicEquation(0)

In [ ]:
planePolars.geodesicEquation(1)

If we replace $\lambda$ with the time $t$, and use dot notation these become

$\begin{align*}
\ddot{r} - r\dot{\theta}^2 &= 0 \\
\ddot{\theta} +\frac{2}{r}\dot{r}\dot{\theta} &= 0
\end{align*}$

If we keep $r$ fixed, we recover the equation for circular motion.

## Some other coordinate systems

### Spherical polars

In [ ]:
sphericalPolars = Metric([r, theta, phi])
sphericalPolars.setDiagonal([1, (r * sin(phi))**2,r**2])

### Cylindricals

In [ ]:
cylindricals = Metric([r, theta, z])
cylindricals.setDiagonal([1,r,1])

In [ ]:
cylindricals.ricci()

In [ ]:
cylindricals.ricciScalar()

### Cylindrical *surface*

In [ ]:
cylindricalSurface = Metric([theta, z])
cylindricalSurface.setDiagonal([r,1])

In [ ]:
cylindricalSurface.ricciScalar()

Yes, a cylindrical surface is flat!

### Schwarzschild

This isn't working properly yet because of the zero'th coordinate being $c t$. I'll think of something..!

In [ ]:
schwarzschild = Metric([t, r, theta, phi])
schwarzschild.setDiagonal([c**2*(1 - (2 * G * M)/(c**2*r) ), -1 / (1 - (2 * G * M)/(c**2*r)), -r**2, - r**2*sin(theta)**2 ])